# **Creating results for 1 dimensional Riemann problems with new loss**
This notebook is for easily creating results to use in the thesis. The code is based on code from the development stage, found in `dev_pkg_newLoss.ipynb` which is a notebook used for testing everything while developing.

## **How to produce results**
1. Choose parameters. Be wise on choice of destination to avoid overwriting.
2. Restart kernel and run all cells.

**NB**: `Make sure not to overwrite wanted material, thus choose destination and filename with care`**!!!**

In [1]:
from datetime import datetime
print("Last run:",datetime.today())

Last run: 2021-03-24 14:48:22.046317


## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from riemannsolver import data1d_newLoss
from riemannsolver import net_mlp1d as network
from riemannsolver import god_mlp1d as god_aprox
from riemannsolver import netplot
from riemannsolver import function
from riemannsolver import godunov as god_exact

from IPython.display import HTML
from tqdm import tqdm_notebook as tqdm

## **Parameters**

#### Flux and derivative

In [3]:
f = lambda u: u**2/2
dfdu = lambda u: u

#### Destination and filename

In [4]:
destination = "res/newLoss/burgers/test"
name = "newloss_"+"test"

#### Training and validation size
* M - number of rows of data (M_val for validation).
* N - mesh size of the two arrays in column direction.
* K - constant to use for variation when creating Fourier coefficients.
* newdata - choose whether or not to create new data.

In [5]:
M = 100000
M_val = M//10
N = 50
K = 50
newdata = False

#### Network parameters

In [6]:
dimensions = (2,600,1)
epochs = 20
batchsize = 100

#### Mesh- and method-parameters

In [7]:
Nx = 50
T = 0.5
C = 0.5

## **Data**

In [8]:
data_train = data1d_newLoss.Dataset(M, N, K, f, dfdu)
data_val = data1d_newLoss.Dataset(M_val, N, K, f, dfdu)
if newdata:
    data_train.create
    data_train.save(destination=destination, filename=name+'_train')
    data_val.create
    data_val.save(destination=destination, filename=name+'_val')
else:
    data_train.load(destination="res/newLoss/burgers/data", filename='100k_train')
    data_val.load(destination="res/newLoss/burgers/data", filename='10k_val')

Data is loaded from res/newLoss/burgers/data/data_100k_train
Data is loaded from res/newLoss/burgers/data/data_10k_val


## **Network**

In [9]:
net = network.Network(
    N=N, 
    dimensions=dimensions,
    dfdu=dfdu,
    activation=F.relu
)

In [10]:
net.backward_newLoss(
    data_train=data_train.data,
    data_val=data_val.data,
    epochs=epochs,
    batchsize=batchsize,
    destination=destination,
    name=name
)
hist = net.history
hist_w = net.history_weight
netplt = netplot.Curve(hist, hist_w)
netplt.plot_history(destination=destination, name=name, show=True)
netplt.plot_weights(destination=destination, name=name, show=True)

RuntimeError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 3.95 GiB total capacity; 1.16 GiB already allocated; 1.07 GiB free; 1.18 GiB reserved in total by PyTorch)

## **Approximate by Godunov**

In [ ]:
u0 = [
    lambda x: (x>=0).type(torch.float64), # Heaviside
    lambda x: (x<0).type(torch.float64), # Reversed Heaviside
    lambda x: ((-1)*(x<0) + (x>0)).type(torch.float64), # Inverse
    lambda x: np.sin(4*np.pi*x) # Trigonometric
]
bounds = ['neumann' for _ in range(3)]+['periodic']
labels = ['heaviside', 'reverseHeavi', 'inverse', 'trig']

for i in range(len(u0)):
    print("Initial function:",labels[i])
    # solve with network
    god_a = god_aprox.Godunov(
        f=f, dfdu=dfdu, u0=u0[i], 
        bnd_cond=bounds[i], 
        xmin=-1, xmax=1, Nx=Nx, 
        network=net.network, T=T, C=C
    )
    god_a.solve
    # solve exact
    god_e = god_exact.Godunov(
        f=f, dfdu=dfdu, u0=u0[i], 
        bnd_cond=bounds[i], 
        xmin=-1, xmax=1, Nx=8192, 
        T=T, C=C
    )
    god_e.solve
    # solve with godunov
    god_g = god_exact.Godunov(
        f=f, dfdu=dfdu, u0=u0[i], 
        bnd_cond=bounds[i], 
        xmin=-1, xmax=1, Nx=Nx, 
        T=T, C=C
    )
    god_g.solve
    # set visuals in netplot module
    netplt.god_net = god_a
    netplt.god_ext = god_e
    netplt.god_god = god_g
    # plot and save
    netplt.plot_solution_exact(destination=destination, name=labels[i]+'_'+name, show=True)
    netplt.plot_solution(destination=destination, name=labels[i]+'_'+name, show=False)
    netplt.plot_exact(destination=destination, name=labels[i]+'_'+name, show=False)
    
    re = float(
        torch.sqrt(torch.sum( (god_a.u[-1]-god_g.u[-1])**2 ))/torch.sqrt(torch.sum( (god_g.u[-1])**2 ))
    )
    print("Relative error - godunov vs network: ",re)